# Analisi dei dati OpenCUP tramite la conversione in LD

In [88]:
import pandas as pd
import rdflib
import json
from IPython.display import Markdown as md
from IPython.display import JSON
import locale
locale.setlocale(locale.LC_ALL, '');


I dati di partenza sono

* I dati di OpenCUP https://www.opencup.gov.it/portale/web/opencup/dettaglio-opendata-complessivo che sono stati convertiti in formato parquet con il tutorial di Andrea Borruso https://aborruso.github.io/posts/csv-ventitre-gigabyte-senza-affanno/
* I dati di IndicePA https://indicepa.gov.it/ipa-dati/ in particolare l'elenco degli enti https://indicepa.gov.it/ipa-dati/dataset/enti
* I dati di PA Digitale 2026 sui progetti finanziati https://github.com/teamdigitale/padigitale2026-opendata/tree/main/data
* I dati di ANAC che connettono CUP e CIG https://dati.anticorruzione.it/opendata/dataset/cup/resource/24991c0c-c091-4505-9a5d-da320be4ebdd
  
Utilizzando duckdb, sono state fatte una serie di join incrociate, che hanno ridotto i dataset soltanto alle parti che effettivamente possono essere collegate fra di loro.

## Conversione da JSON a JSON-LD

Per la conversione da JSON a JSON-LD sono stati utilizzati dei template in formato handlebars.js (https://handlebarsjs.com/) per alcuni motivi:
* la versatilità: ci sono varie librerie che implementano la conversione
* la possibilità di essere letti anche da una persona
* la possibilità di modificare la struttura del JSON di partenza

Scegliamo di utilizzare un approccio incrementale per la conversione, in modo da poter verificare la bontà dei collegamenti

In [89]:
from pybars import Compiler
import urllib.parse

# Funzione helper per evitare di ottenere IRI non valide
def encodeURIComponent(this,value):
    return urllib.parse.quote(value)

def apply_handlebars_template(template, data):
    compiler = Compiler()
    context = {
        'encodeURIComponent': encodeURIComponent
    }
    return compiler.compile(template)(data, helpers=context)


### Dati di OpenCUP

[Conversione dati di OpenCUP](ConvDatiOpenCUP.ipynb)


### Dati di IndicePA

[Conversione dati di IndicePA](ConvDatiIndicePA.ipynb)


### Dati di PA Digitale

[Conversione dati di PA Digitale](ConvDatiPADigitale.ipynb)


## Caricamento dati e analisi

In [90]:
opencup_g = rdflib.Graph()
opencup_g.parse("LD/json-ld/opencup_candidature_comuni_finanziate-ld.json",format="json-ld");

In [91]:
indicePA_g = rdflib.Graph()
indicePA_g.parse("LD/json-ld/candidature_enti_ipa-ld.json",format="json-ld");

In [92]:
PADigitale_g = rdflib.Graph()
PADigitale_g.parse("LD/json-ld/candidature_comuni_finanziate-ld.json",format="json-ld");

In [93]:
all_g = opencup_g + indicePA_g + PADigitale_g

In [94]:
#all_g.print()

## Query: costo totale progetti per ente

In [95]:
query="""
SELECT ?funderName (SUM(?totalCost) AS ?funderTotalCost)
WHERE {
    ?prj a PRJ:PublicInvestmentProject;
    PRJ:projectFunder ?funder;
    PRJ:projectTotalCost ?totalCost.
    ?funderOrg a COV:PublicOrganization;
    owl:sameAs ?funder;
    COV:legalName ?funderName.
    
}
GROUP BY ?funder
ORDER BY DESC(?funderTotalCost)
"""

res=all_g.query(query)

for row in res:
    print(row.funderName, row.funderTotalCost)

Comune di Venezia 7127677
Comune di Bari 4858680
Comune di Ferrara 1692369
Comune di Messina 1547897
Comune di Piacenza 1479027
Comune di Rimini 1031574
Comune di Olbia 965270
Comune di San Benedetto del Tronto 752741
Comune di Trento 719758
Comune di Mascalucia 679976
Comune di Oderzo 665866
Comune di Bussolengo 643416
Comune di Desio 643416
Comune di Modena 616095
Comune di Ancona 614609
Comune di Castiglione delle Stiviere 600427
Comune di Bastia Umbra 587589
Comune di Salerno 585323
Comune di Mantova 564156
Comune di Vicenza 554781
Comune di Giussano 533050
Comune di Conegliano 527756
Comune di La Spezia 527308
Comune di Latina 516323
Comune di Treviso 419124
Comune di Rho 419124
Comune di Castelfranco Emilia 407813
Comune di Calcio 404159
Comune di Vigevano 402506
Comune di Villa Guardia 392385
Comune di Castano Primo 376098
Comune di Castel San Pietro Terme 371413
Comune di Terno d&#x27;Isola 371305
Comune di Galbiate 365895
Comune di Mesola 357000
Comune di Lugo 355151
Comune di

## Query: costo totale progetti per ente e misura di PA Digitale

In [96]:
query="""
SELECT ?callName ?funderName (SUM(?totalCost) AS ?funderTotalCost)
WHERE {
    ?prj a PRJ:PublicInvestmentProject;
    PRJ:projectFunder ?funder;
    PRJ:projectTotalCost ?totalCost;
    PRJ:hasCall ?call.
    ?funderOrg a COV:PublicOrganization;
    owl:sameAs ?funder;
    COV:legalName ?funderName.
    ?call l0:name ?callName.
}
GROUP BY ?funderName ?callName
ORDER BY DESC(?funderTotalCost)
"""

res=all_g.query(query)

for row in res:
    print("{}:{}: {:n}".format(row.funderName, row.callName, int(row.funderTotalCost)))

Comune di Venezia:1.2 Abilitazione al cloud - Comuni - Aprile 2022: 5.092.136
Comune di Bari:1.2 Abilitazione al Cloud - Comuni - Luglio 2022: 4.858.680
Comune di Bari:1.2 Abilitazione al Cloud - Comuni - novembre 2023: 4.858.680
Comune di Venezia:1.4.1 Esperienza del Cittadino - Comuni - Settembre 2022: 1.277.083
Comune di Rimini:1.2 Abilitazione al cloud - Comuni - Aprile 2022: 1.031.574
Comune di Messina:1.2 Abilitazione al Cloud - Comuni - Luglio 2022: 1.031.574
Comune di Ferrara:1.2 Abilitazione al cloud - Comuni - Aprile 2022: 1.031.574
Comune di Piacenza:1.2 Abilitazione al cloud - Comuni - Aprile 2022: 797.874
Comune di Messina:1.4.1 Esperienza del Cittadino - Comuni - Aprile 2022: 516.323
Comune di Modena:1.4.1 Esperienza del Cittadino - Comuni - Aprile 2022: 516.323
Comune di Latina:1.4.1 Esperienza del Cittadino - Comuni - Aprile 2022: 516.323
Comune di Piacenza:1.4.1 Esperienza del Cittadino - Comuni - Settembre 2022: 516.323
Comune di Vicenza:1.4.1 Esperienza del Cittadino

In [97]:
query="""
SELECT ?callName (SUM(?totalCost) AS ?funderTotalCost)
WHERE {
    ?prj a PRJ:PublicInvestmentProject;
    PRJ:projectFunder ?funder;
    PRJ:projectTotalCost ?totalCost;
    PRJ:hasCall ?call.
    ?funderOrg a COV:PublicOrganization;
    owl:sameAs ?funder;
    COV:legalName ?funderName.
    ?call l0:name ?callName.
}
GROUP BY ?callName
ORDER BY DESC(?funderTotalCost)
"""

res=all_g.query(query)

for row in res:
    print("{} : {:n}".format(row.callName, int(row.funderTotalCost)))

1.2 Abilitazione al cloud - Comuni - Aprile 2022 : 19.694.472
1.4.1 Esperienza del Cittadino - Comuni - Aprile 2022 : 15.382.425
1.2 Abilitazione al Cloud - Comuni - Luglio 2022 : 9.718.759
1.4.1 Esperienza del Cittadino - Comuni - Settembre 2022 : 9.610.781
1.2 Abilitazione al Cloud - Comuni - novembre 2023 : 5.447.223
1.3.1 Piattaforma Digitale Nazionale Dati - Comuni Ottobre 2022 : 2.594.038
1.4.5 &#x27;Piattaforma Notifiche Digitali&#x27; Comuni (Settembre 2022) : 2.365.235
1.4.3 - app IO - Comuni - Aprile 2022 : 1.028.360
1.4.3 - pagoPA - Comuni - Aprile 2022 : 1.004.768
1.4.3 - pagoPA - Comuni - Settembre 2022 : 735.966
1.4.4 Adozione SPID CIE  - Comuni - Aprile 2022 : 420.000
1.4.3 - pagoPA - Comuni - maggio 2023 : 224.093
1.4.4 Adozione SPID CIE  - Comuni - Settembre 2022 : 126.000
1.4.3 - app IO - Comuni - Settembre 2022 : 50.980
